In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore")

In [10]:
h1 = {
'Cookie': 'bid=nci9lEBl2e8; _ga=GA1.2.1430306498.1676346220; douban-fav-remind=1; dbcl2="276029126:0x9DF7sHFuc"; push_noty_num=0; push_doumail_num=0; __utmv=30149280.27602; ll="108090"; _pk_id.100001.afe6=19ddc509b69de8c8.1700653601.; _vwo_uuid_v2=D7EE642712B5437140F7E64A1B44F0556|f7465dfd914c75b4d14ad2fa3db609c4; Hm_lvt_16a14f3002af32bf3a75dfe352478639=1700733756; __utma=266659602.1430306498.1676346220.1700733771.1700733771.1; __utmz=266659602.1700733771.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); ck=VjRN; ap_v=0,6.0; frodotk_db="b901b976117a702f2b052c62346f508d"; __utma=30149280.1430306498.1676346220.1700737591.1700831159.10; __utmc=30149280; __utmz=30149280.1700831159.10.5.utmcsr=music.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/subject/30283109/; _pk_ref.100001.afe6=%5B%22%22%2C%22%22%2C1700831164%2C%22https%3A%2F%2Fsearch.douban.com%2Fmusic%2Fsubject_search%3Fsearch_text%3Dtaylor%26cat%3D1003%22%5D; _pk_ses.100001.afe6=1; __utmt=1; __utmb=30149280.20.10.1700831159',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
'Accept-Encoding': 'gzip, deflate, br',
'Host': 'music.douban.com',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0',
'Accept-Language': 'h-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
'Connection': 'keep-alive'
}

In [3]:
ip_list = []

with open('proxy.txt', 'r') as f:
    for line in f.readlines():
        ip_list.append(line.strip())

#获取ip代理
def getip():
    proxy= ip_list[random.randint(0,len(ip_list)-1)]
    proxy = proxy.replace("\n","")
    proxies={
        'http':'http://'+str(proxy),
        #'https':'https://'+str(proxy),
    }
    return proxies

In [4]:
def trans_star(v_str):
    if(len(v_str)==1):
        return 'NaN'
    v_str = v_str[1]
    if v_str == 'allstar10':
        return '1'
    elif v_str == 'allstar20':
        return '2'
    elif v_str == 'allstar30':
        return '3'
    elif v_str == 'allstar40':
        return '4'
    elif v_str == 'allstar50':
        return '5'
    else:
        return 'NaN'

In [5]:
def get_short(music_url, max_page, data_path, file_name):
    df = pd.DataFrame()
    path =data_path + file_name + '.csv'
    if os.path.exists(path):
        print("file exists")
        return 0
    for i in range(0, max_page):
        url = music_url + 'comments?start=' + str(i*20)+ '&limit=20&status=P&sort=new_score'
        res = requests.get(url, headers=h1, verify=False, proxies=getip(), timeout=10)
        #res = requests.get(url, headers=h1, verify=False, proxies=getip() ,allow_redirects=False)
        
        soup = BeautifulSoup(res.text, 'html.parser')
        
        user_names = soup.find_all('span', class_='comment-info')
        user_names = [i.find('a').text for i in user_names]
        
        stars = soup.find_all('span', class_='comment-info')
        stars = [i.find_all('span')[0]['class'] for i in stars]
        stars = [trans_star(i) for i in stars]
        
        comment_times = soup.find_all('span', class_='comment-time')
        comment_times = [i.text.strip() for i in comment_times]
        
        votes = soup.find_all('span', class_='vote-count')
        votes = [i.text for i in votes]
        
        shorts = soup.find_all('span', class_='short')
        shorts = [i.text for i in shorts]
        shorts = [i.replace(',', '，').replace(' ', '').replace('\n', '').replace('\r', '') for i in shorts]
        
        pd_data = {'user_name': user_names, 'star': stars, 'comment_time': comment_times, 'vote': votes, 'short': shorts}
        print(len(user_names),len(stars), len(comment_times), len(votes), len(shorts))
        df = pd.concat([df, pd.DataFrame(pd_data)], axis=0)
        print('Page ' + str(i + 1) + ' is done!')
        sleep(random.uniform(0,2))
        #if i % 5 == 0:
            #sleep(random.uniform(1,2))
            #print('Page ' + str(i + 1) + ' is done!')

    if len(df) < 10:
        print(len(df),': No data!')
        return 1

    df.to_csv(path, index=False)
    return 0

In [8]:
#test
music_url = 'https://music.douban.com/subject/35999301/comments?start=0&limit=20&status=P&sort=new_score'
max_page = 20
data_path = './data/'
file_name = '启示录'
get_short(music_url, max_page, data_path, file_name)

20 20 20 20 20
Page 1 is done!
20 20 20 20 20
Page 2 is done!
20 20 20 20 20
Page 3 is done!
20 20 20 20 20
Page 4 is done!
20 20 20 20 20
Page 5 is done!
20 20 20 20 20
Page 6 is done!
20 20 20 20 20
Page 7 is done!
20 20 20 20 20
Page 8 is done!
20 20 20 20 20
Page 9 is done!
20 20 20 20 20
Page 10 is done!
20 20 20 20 20
Page 11 is done!
20 20 20 20 20
Page 12 is done!
20 20 20 20 20
Page 13 is done!
20 20 20 20 20
Page 14 is done!
20 20 20 20 20
Page 15 is done!
20 20 20 20 20
Page 16 is done!
20 20 20 20 20
Page 17 is done!
20 20 20 20 20
Page 18 is done!
20 20 20 20 20
Page 19 is done!
20 20 20 20 20
Page 20 is done!


0

In [6]:
# read movie list
music_list = pd.read_csv('../analysis/musicDouban250.csv')
# each line as a movie, convert to list of dict
music_list = music_list.to_dict('records')
#print(music_list)


In [40]:
music_list

[{'Unnamed: 0': 0,
  '专辑名称': 'We Sing. We Dance. We Steal Things.',
  '专辑链接': 'https://music.douban.com/subject/2995812/',
  '专辑评分': 9.1,
  '评分人数': 116183,
  '歌手': 'Jason Mraz',
  '发行日期': '2008-05-13',
  '类型': 'Import',
  '介质': 'Audio CD',
  '曲风': '民谣'},
 {'Unnamed: 0': 1,
  '专辑名称': 'Viva La Vida Death And All His Friends',
  '专辑链接': 'https://music.douban.com/subject/3040149/',
  '专辑评分': 9.0,
  '评分人数': 118408,
  '歌手': 'Coldplay',
  '发行日期': '2008-06-17',
  '类型': '专辑',
  '介质': 'CD',
  '曲风': '摇滚'},
 {'Unnamed: 0': 2,
  '专辑名称': '华丽的冒险 華麗的冒險',
  '专辑链接': 'https://music.douban.com/subject/1427374/',
  '专辑评分': 9.0,
  '评分人数': 89971,
  '歌手': '陈绮贞',
  '发行日期': '2005-09-23',
  '类型': '专辑',
  '介质': 'CD',
  '曲风': '流行'},
 {'Unnamed: 0': 3,
  '专辑名称': '范特西 Fantasy',
  '专辑链接': 'https://music.douban.com/subject/1403307/',
  '专辑评分': 9.5,
  '评分人数': 177661,
  '歌手': '周杰伦',
  '发行日期': '2001-09-14',
  '类型': '专辑',
  '介质': 'CD',
  '曲风': '流行'},
 {'Unnamed: 0': 4,
  '专辑名称': '後。青春期的詩 后青春期的诗',
  '专辑链接': 'https://music.

In [11]:
music_url = 'https://music.douban.com/subject/35999301/'
max_page = 20
data_path = './data/'
file_name ='启示录'
ret = get_short(music_url, max_page, data_path, file_name)
if ret == 1:
    print("blocked!")

20 20 20 20 20
Page 1 is done!
20 20 20 20 20
Page 2 is done!
20 20 20 20 20
Page 3 is done!
20 20 20 20 20
Page 4 is done!
20 20 20 20 20
Page 5 is done!
20 20 20 20 20
Page 6 is done!
20 20 20 20 20
Page 7 is done!
20 20 20 20 20
Page 8 is done!
20 20 20 20 20
Page 9 is done!
20 20 20 20 20
Page 10 is done!
20 20 20 20 20
Page 11 is done!
0 0 0 0 0
Page 12 is done!
0 0 0 0 0
Page 13 is done!
0 0 0 0 0
Page 14 is done!
0 0 0 0 0
Page 15 is done!
0 0 0 0 0
Page 16 is done!
0 0 0 0 0
Page 17 is done!
0 0 0 0 0
Page 18 is done!
0 0 0 0 0
Page 19 is done!
0 0 0 0 0
Page 20 is done!


In [13]:
# get short for each movie
for music in music_list:
    music_url = music['专辑链接']
    max_page = 5
    data_path = './data/'
    file_name = music['专辑名称'].replace(' ', '').split('/')[0].replace('\xa0', '').replace('?','')
    ret = get_short(music_url, max_page, data_path, file_name)
    if ret == 1:
        print("blocked!")
        break
    print(music['专辑名称'] + ' is done!')
##Once (Soundtrack) Once这个只有25页


file exists
We Sing. We Dance. We Steal Things. is done!
file exists
Viva La Vida Death And All His Friends is done!
file exists
华丽的冒险 華麗的冒險 is done!
file exists
范特西 Fantasy is done!
file exists
後。青春期的詩 后青春期的诗 is done!
file exists
是时候 It's Time is done!
file exists
Lenka is done!
file exists
Start from Here 从这里开始 is done!
file exists
旅行的意义 is done!
file exists
太阳 Immortal is done!
file exists
Once (Soundtrack) Once / 电影《曾经》原声大碟 is done!
file exists
Not Going Anywhere 守候 is done!
file exists
American Idiot is done!
file exists
思念是一种病 OK is done!
file exists
無與倫比的美麗 无与伦比的美丽 is done!
file exists
亲爱的...我还不知道 親愛的…我還不知道 is done!
file exists
城市 The City is done!
file exists
O is done!
file exists
Wake Me Up When September Ends 九月结束的时候叫醒我 is done!
file exists
叶惠美 葉惠美 / Yeh Hui-Mei is done!
file exists
七里香 Common Jasmin Orange is done!
file exists
21 is done!
file exists
My Life Will... is done!
file exists
寓言 is done!
file exists
你在煩惱什麼 你在烦恼什么 is done!
file exists
感官/世界 Senses Around is done!


TooManyRedirects: Exceeded 30 redirects.